<a href="https://colab.research.google.com/github/asirkov/weapon-detection/blob/main/Weapon_Detection_Mask_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/matterport/Mask_RCNN

In [ ]:
import shutil, os  

# Copy required files

if os.path.exists("/content/mrcnn"):
    shutil.rmtree("/content/mrcnn")

if not os.path.exists("/content/mrcnn"):
  shutil.copytree('/content/drive/MyDrive/Mask_RCNN/mrcnn','/content/mrcnn')


# Remove loaded directory
if os.path.exists("/content/Mask_RCNN"):
  shutil.rmtree("/content/Mask_RCNN")
  
# Copy start COCO weights
if not os.path.exists("/content/weights.h5"):
  shutil.copy2('/content/drive/MyDrive/Mask_RCNN/mask_rcnn_coco.h5','/content/weights.h5')
  
# # Copy weapon weights
# if not os.path.exists("/content/weights.h5"):
  # shutil.copy2('/content/drive/MyDrive/Mask_RCNN/logs4/mask_rcnn_object_0050.h5','/content/weights.h5')

if os.path.exists("/content/dataset"):
  shutil.rmtree("/content/dataset")

# Copy dataset
if not os.path.exists("/content/dataset"):
  shutil.copytree('/content/drive/MyDrive/Mask_RCNN/dataset3','/content/dataset')

In [ ]:
!pip install keras==2.2.5
%tensorflow_version 1.x

!pip install h5py==2.10.0

In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

# Set root directory of the project
ROOT_DIR = os.path.abspath("/content/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
WEIGHTS_PATH = os.path.join(ROOT_DIR, "weights.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
# Create configuration class

class CustomConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 4  # Background + knife + pistol + carabine + rifle

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
# Create dataset class

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "knife")
        self.add_class("object", 2, "pistol")
        self.add_class("object", 3, "carabine")
        self.add_class("object", 4, "rifle")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open(os.path.join(dataset_dir, "annotations.json")))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['type'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"knife": 1, "pistol": 2, "carabine": 3, "rifle": 4}
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
# Create train method

def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("/content/dataset", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("/content/dataset", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=100,
                layers='heads')

In [ ]:
# Create config and model objects

config = CustomConfig()
model = modellib.MaskRCNN(mode="training", 
                          config=config,
                          model_dir=DEFAULT_LOGS_DIR)

In [ ]:
# Optional: download last version of coco_weights

# Download weights file
# if not os.path.exists(COCO_WEIGHTS_PATH):
  # utils.download_trained_weights(COCO_WEIGHTS_PATH)

In [ ]:
# Load weights into model

model.load_weights(WEIGHTS_PATH, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", 
                            "mrcnn_bbox_fc",
                            "mrcnn_bbox", 
                            "mrcnn_mask"])

In [ ]:
# Start traning

train(model)


In [ ]:
import shutil

# Load weights logs
shutil.copytree('/content/logs', '/content/drive/MyDrive/Mask_RCNN/logs5')